# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-26-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-26-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-08-27 04:21:37,33.93911,67.709953,152822,7093,NaN,NaN,Afghanistan,392.572599,4.641347
1,NaN,NaN,NaN,Albania,2021-08-27 04:21:37,41.15330,20.168300,142253,2486,NaN,NaN,Albania,4943.116269,1.747591
2,NaN,NaN,NaN,Algeria,2021-08-27 04:21:37,28.03390,1.659600,193674,5121,NaN,NaN,Algeria,441.663383,2.644134
3,NaN,NaN,NaN,Andorra,2021-08-27 04:21:37,42.50630,1.521800,15016,130,NaN,NaN,Andorra,19434.414030,0.865743
4,NaN,NaN,NaN,Angola,2021-08-27 04:21:37,-11.20270,17.873900,46726,1182,NaN,NaN,Angola,142.170081,2.529641


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/17/21,8/18/21,8/19/21,8/20/21,8/21/21,8/22/21,8/23/21,8/24/21,8/25/21,8/26/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,152243,152363,152411,152448,152448,152448,152583,152660,152722,152822
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,136598,137075,137597,138132,138790,139324,139721,140521,141365,142253
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,188663,189384,190078,190656,191171,191583,192089,192626,193171,193674


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/17/21,8/18/21,8/19/21,8/20/21,8/21/21,8/22/21,8/23/21,8/24/21,8/25/21,8/26/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7031,7043,7047,7054,7054,7054,7076,7083,7090,7093
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2468,2471,2473,2475,2477,2478,2478,2480,2483,2486
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,4867,4898,4929,4957,4984,5004,5034,5063,5096,5121


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/17/21,8/18/21,8/19/21,8/20/21,8/21/21,8/22/21,8/23/21,8/24/21,8/25/21,8/26/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/17/21,8/18/21,8/19/21,8/20/21,8/21/21,8/22/21,8/23/21,8/24/21,8/25/21,8/26/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,8091,8132,8162,8199,8286,8311,8332,8332,8332,8486
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,29465,29836,30106,30484,30939,31176,31313,31313,31313,31878
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2785,2805,2821,2833,2851,2872,2878,2878,2878,2929


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/17/21,8/18/21,8/19/21,8/20/21,8/21/21,8/22/21,8/23/21,8/24/21,8/25/21,8/26/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,115,115,115,115,115,115,115,115,115,115
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,350,352,355,358,364,364,364,364,364,369
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,65,65,65,65,65,65,65,65,65,65


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
656,1001.0,Autauga,Alabama,US,2021-08-27 04:21:37,32.539527,-86.644082,8486,115,NaN,NaN,"Autauga, Alabama, US",15189.103080,1.355173
693,1075.0,Lamar,Alabama,US,2021-08-27 04:21:37,33.779950,-88.096680,1675,38,NaN,NaN,"Lamar, Alabama, US",12133.285042,2.268657
694,1077.0,Lauderdale,Alabama,US,2021-08-27 04:21:37,34.901719,-87.656247,11599,258,NaN,NaN,"Lauderdale, Alabama, US",12508.492489,2.224330


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,38384360,633564,0.0
India,32558530,436365,0.0
Brazil,20676561,577565,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,38384360,633564,0.0,37750796.0,2021-08-27 04:21:37,37.950547,-91.419547
India,32558530,436365,0.0,32122165.0,2021-08-27 04:21:37,23.088275,81.806127
Brazil,20676561,577565,0.0,20098996.0,2021-08-27 04:21:37,-12.669522,-48.480493
France,6777676,114269,0.0,6663407.0,2021-08-27 04:21:37,6.430808,-34.730285
Russia,6728699,176127,0.0,6552572.0,2021-08-27 04:21:37,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,4293363,65508,0.0
Texas,3534862,56055,0.0
Florida,3071489,42252,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,4293363,65508,0.0,4227855.0,2021-08-27 04:21:37,37.843962,-120.728594
Texas,3534862,56055,0.0,3478807.0,2021-08-27 04:21:37,31.660643,-98.653069
Florida,3071489,42252,0.0,3029237.0,2021-08-27 04:21:37,28.940755,-82.700744
New York,2254155,54130,0.0,2200025.0,2021-08-27 04:21:37,42.544151,-75.474183
Illinois,1503063,26308,0.0,1476755.0,2021-08-27 04:21:37,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1394632,25182,0.0
Arizona,Maricopa,631156,10705,0.0
Florida,Miami-Dade,610802,6472,0.0
Illinois,Cook,586687,10727,0.0
Texas,Harris,486701,7091,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1394632,25182,0.0,1369450.0,2021-08-27 04:21:37,34.308284,-118.228241,6037.0
Arizona,Maricopa,631156,10705,0.0,620451.0,2021-08-27 04:21:37,33.348359,-112.491815,4013.0
Florida,Miami-Dade,610802,6472,0.0,604330.0,2021-08-27 04:21:37,25.611236,-80.551706,12086.0
Illinois,Cook,586687,10727,0.0,575960.0,2021-08-27 04:21:37,41.841448,-87.816588,17031.0
Texas,Harris,486701,7091,0.0,479610.0,2021-08-27 04:21:37,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-08-24,152660,140521,192626,15003,46340,1540,5148085,238422,46725,678605,...,6586181,384287,150683,4,327450,369267,329204,7580,204977,123320
2021-08-25,152722,141365,193171,15014,46539,1598,5155079,239056,47840,680179,...,6621799,384392,151517,4,328550,381363,330821,7625,205107,123711
2021-08-26,152822,142253,193674,15016,46726,1598,5161926,239739,48816,681617,...,6659916,384458,152363,4,329736,392938,332609,7676,205315,123986


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-08-24,7083,2480,5063,130,1166,43,110806,4768,986,10768,...,132174,6020,1035,1,3924,9014,3646,1424,3583,4320
2021-08-25,7090,2483,5096,130,1176,43,110966,4778,989,10770,...,132323,6022,1043,1,3940,9349,3654,1438,3586,4338
2021-08-26,7093,2486,5121,130,1182,43,111117,4785,991,10772,...,132465,6025,1050,1,3955,9667,3657,1444,3587,4356


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-08-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-08-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-08-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/17/21,8/18/21,8/19/21,8/20/21,8/21/21,8/22/21,8/23/21,8/24/21,8/25/21,8/26/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,8091,8132,8162,8199,8286,8311,8332,8332,8332,8486
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1595,1610,1618,1626,1645,1648,1652,1652,1652,1675
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,11064,11147,11195,11281,11371,11395,11434,11434,11434,11599
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3433,3460,3483,3501,3540,3550,3569,3569,3569,3629
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,18623,18759,18875,18994,19198,19291,19342,19342,19342,19577


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-08-24,665653,84867,0,991309,438465,4266957,604516,368481,117246,49,...,1002632,3499482,455513,27237,5684,744187,539887,180804,720460,72069
2021-08-25,665653,85491,0,991309,441246,4280215,606236,369132,117587,49,...,1002632,3520818,457098,27355,5707,747641,542878,181906,722453,72528
2021-08-26,676795,86218,0,998164,443564,4293363,608047,369920,118016,49,...,1013943,3534862,458589,27504,5760,751132,546721,183354,724717,72995


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-08-24        8332   31313    2878  3306   8173    1346   2694   16949   
2021-08-25        8332   31313    2878  3306   8173    1346   2694   16949   
2021-08-26        8486   31878    2929  3359   8340    1359   2747   17154   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-08-24         4500     2290  ...    3288    890     3681      909   
2021-08-25         4500     2290  ...    3309    895     3737      912   
2021-08-26         4540     2318  ...    3336    907     3762      915   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-08-24           5345  4265  2810          0      997    757  
2021-08-25           5378  4300  2841          0     1003    760  
2021-08-26           5406  4329  2873          0     1014    765  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-08-24,12000,427,0,18638,6749,65259,7084,8335,1857,0,...,13228,55549,2593,270,51,11693,6417,3017,8406,835
2021-08-25,12000,438,0,18638,6774,65399,7088,8335,1869,0,...,13228,55798,2605,271,51,11715,6448,3036,8416,835
2021-08-26,12103,438,0,18661,6806,65508,7095,8355,1872,0,...,13296,56055,2615,273,53,11729,6471,3049,8429,835


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-08-24         115     364      65   69    142      41     72     350   
2021-08-25         115     364      65   69    142      41     72     350   
2021-08-26         115     369      65   69    142      41     73     350   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-08-24          126       49  ...      37     15       33       10   
2021-08-25          126       49  ...      37     15       33       10   
2021-08-26          127       49  ...      37     15       33       10   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-08-24             49    11    14          0       26      6  
2021-08-25             49    11    14          0       26      6  
2021-08-26             49    11    14          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-08-24,0.000505,0.005726,0.002796,0.000067,0.005730,0.000000,0.001580,0.002257,0.021222,0.001479,...,0.004693,0.000276,0.005384,0.0,0.002842,0.030160,0.004792,0.005438,0.001593,0.002593
2021-08-25,0.000406,0.006006,0.002829,0.000733,0.004294,0.037662,0.001359,0.002659,0.023863,0.002319,...,0.005408,0.000273,0.005535,0.0,0.003359,0.032757,0.004912,0.005937,0.000634,0.003171
2021-08-26,0.000655,0.006282,0.002604,0.000133,0.004018,0.000000,0.001328,0.002857,0.020401,0.002114,...,0.005756,0.000172,0.005584,0.0,0.003610,0.030352,0.005405,0.006689,0.001014,0.002223


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-08-24,0.000989,0.000807,0.005761,0.0,0.002580,0.0,0.001781,0.001260,0.002033,0.000465,...,0.001318,0.000665,0.006809,0.0,0.003837,0.040157,0.001098,0.002817,0.001397,0.006289
2021-08-25,0.000988,0.001210,0.006518,0.0,0.008576,0.0,0.001444,0.002097,0.003043,0.000186,...,0.001127,0.000332,0.007729,0.0,0.004077,0.037164,0.002194,0.009831,0.000837,0.004167
2021-08-26,0.000423,0.001208,0.004906,0.0,0.005102,0.0,0.001361,0.001465,0.002022,0.000186,...,0.001073,0.000498,0.006711,0.0,0.003807,0.034014,0.000821,0.004172,0.000279,0.004149


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-08-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-08-24,0.000000,0.007060,NaN,0.002625,0.005096,0.003588,0.002072,0.002915,0.001666,0.0,...,0.005166,0.006641,0.002509,0.003870,0.008159,0.004084,0.005725,0.004361,0.003551,0.007085
2021-08-25,0.000000,0.007353,NaN,0.000000,0.006343,0.003107,0.002845,0.001767,0.002908,0.0,...,0.000000,0.006097,0.003480,0.004332,0.004046,0.004641,0.005540,0.006095,0.002766,0.006369
2021-08-26,0.016738,0.008504,NaN,0.006915,0.005253,0.003072,0.002987,0.002135,0.003648,0.0,...,0.011281,0.003989,0.003262,0.005447,0.009287,0.004669,0.007079,0.007960,0.003134,0.006439


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-08-24      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2021-08-25      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2021-08-26      0.018483  0.018044  0.017721  0.016031  0.020433  0.009658   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-08-24      0.000000  0.000000  0.000000  0.000000  ...  0.006428   
2021-08-25      0.000000  0.000000  0.000000  0.000000  ...  0.006387   
2021-08-26      0.019673  0.012095  0.008889  0.012227  ...  0.008160   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-08-24      0.004515  0.007114  0.007761   0.003191  0.003057  0.013343   
2021-08-25      0.005618  0.015213  0.003300   0.006174  0.008206  0.011032   
2021-08-26      0.013408  0.006690  0.003289   0.005206  0.006744  0.011264   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-08-24            NaN  0.002010  0.007989  
2021-08-25            NaN  0.006018  0.003963  
2021-08-26            NaN  0.010967  0.006579  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-08-24,0.000000,0.031401,NaN,0.002043,0.006712,0.001366,0.000848,0.0000,0.001618,NaN,...,0.002349,0.003668,0.003483,0.000000,0.040816,0.001542,0.005327,0.000332,0.003941,0.032138
2021-08-25,0.000000,0.025761,NaN,0.000000,0.003704,0.002145,0.000565,0.0000,0.006462,NaN,...,0.000000,0.004483,0.004628,0.003704,0.000000,0.001881,0.004831,0.006298,0.001190,0.000000
2021-08-26,0.008583,0.000000,NaN,0.001234,0.004724,0.001667,0.000988,0.0024,0.001605,NaN,...,0.005141,0.004606,0.003839,0.007380,0.039216,0.001195,0.003567,0.004282,0.001545,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                  \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock    Butler   
2021-08-24         0.0  0.000000     0.0  0.0    0.0     0.0  0.000000   
2021-08-25         0.0  0.000000     0.0  0.0    0.0     0.0  0.000000   
2021-08-26         0.0  0.013736     0.0  0.0    0.0     0.0  0.013889   

Province_State                             ...   Wyoming                  \
Admin2         Calhoun  Chambers Cherokee  ...      Park Platte Sheridan   
2021-08-24         0.0  0.000000      0.0  ...  0.057143    0.0      0.0   
2021-08-25         0.0  0.000000      0.0  ...  0.000000    0.0      0.0   
2021-08-26         0.0  0.007937      0.0  ...  0.000000    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-08-24          0.0   0.065217   0.0   0.0        NaN      0.0    0.0  
2021-08-25          0.0   0.000000   0.0   0.0        NaN      0.0    0.0  
2021-08-26          0.0   0.000000   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-08-24,0.000501,0.004583,0.002708,0.000301,0.004544,0.010969,0.001340,0.001862,0.020288,0.001579,...,0.004838,0.000258,0.005528,3.918315e-39,0.003358,0.031468,0.004110,0.004691,0.001281,0.002595
2021-08-25,0.000453,0.005295,0.002769,0.000517,0.004419,0.024316,0.001349,0.002260,0.022076,0.001949,...,0.005123,0.000266,0.005531,1.959157e-39,0.003359,0.032112,0.004511,0.005314,0.000958,0.002883
2021-08-26,0.000554,0.005788,0.002686,0.000325,0.004219,0.012158,0.001339,0.002559,0.021238,0.002032,...,0.005440,0.000219,0.005557,9.795786e-40,0.003484,0.031232,0.004958,0.006001,0.000986,0.002553


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-08-24,0.001339,0.000558,0.005605,0.001938,0.005368,5.543587e-12,0.001833,0.001708,0.001761,0.000275,...,0.000888,0.000365,0.006707,0.0,0.003999,0.047450,0.001135,0.002071,0.001198,0.006849
2021-08-25,0.001164,0.000884,0.006062,0.000969,0.006972,2.771793e-12,0.001639,0.001903,0.002402,0.000230,...,0.001008,0.000349,0.007218,0.0,0.004038,0.042307,0.001665,0.005951,0.001017,0.005508
2021-08-26,0.000793,0.001046,0.005484,0.000485,0.006037,1.385897e-12,0.001500,0.001684,0.002212,0.000208,...,0.001040,0.000423,0.006965,0.0,0.003923,0.038161,0.001243,0.005062,0.000648,0.004829


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-08-24,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.481849,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,-0.497938,-0.498727,-0.489078
2021-08-25,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.481849,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,-0.497938,-0.498727,-0.489078
2021-08-26,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.481849,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,-0.497938,-0.498727,-0.489078


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-08-24,0.002575,0.007307,NaN,0.002802,0.004222,0.003493,0.003018,0.002611,0.002179,1.549688e-159,...,0.008189,0.005140,0.002927,0.003938,0.007024,0.004689,0.007264,0.006016,0.003091,0.007095
2021-08-25,0.001288,0.007330,NaN,0.001401,0.005282,0.003300,0.002932,0.002189,0.002544,7.748442e-160,...,0.004094,0.005619,0.003203,0.004135,0.005535,0.004665,0.006402,0.006055,0.002929,0.006732
2021-08-26,0.009013,0.007917,NaN,0.004158,0.005268,0.003186,0.002959,0.002162,0.003096,3.874221e-160,...,0.007688,0.004804,0.003233,0.004791,0.007411,0.004667,0.006741,0.007008,0.003031,0.006585


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-08-24      0.001948  0.003682  0.002171  0.003320  0.002658  0.001536   
2021-08-25      0.000974  0.001841  0.001086  0.001660  0.001329  0.000768   
2021-08-26      0.009729  0.009942  0.009403  0.008846  0.010881  0.005213   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-08-24      0.003604  0.002367  0.002736  0.004360  ...  0.007168   
2021-08-25      0.001802  0.001183  0.001368  0.002180  ...  0.006778   
2021-08-26      0.010738  0.006639  0.005128  0.007204  ...  0.007469   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-08-24      0.007553  0.007494  0.004763   0.004543  0.006262  0.010866   
2021-08-25      0.006586  0.011353  0.004032   0.005358  0.007234  0.010949   
2021-08-26      0.009997  0.009022  0.003661   0.005282  0.006989  0.011106   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-08-24           -1.0  0.003736  0.009262  
2021-08-25           -1.0  0.004877  0.006613  
2021-08-26           -1.0  0.007922  0.006596  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-08-24,0.000474,0.015978,NaN,0.001273,0.005585,0.001039,0.000873,0.000011,0.000970,NaN,...,0.002559,0.002648,0.003151,0.000381,0.045196,0.001415,0.003988,0.000960,0.002193,0.016148
2021-08-25,0.000237,0.020870,NaN,0.000637,0.004645,0.001592,0.000719,0.000006,0.003716,NaN,...,0.001280,0.003565,0.003889,0.002042,0.022598,0.001648,0.004409,0.003629,0.001691,0.008074
2021-08-26,0.004410,0.010435,NaN,0.000935,0.004684,0.001629,0.000853,0.001203,0.002661,NaN,...,0.003210,0.004086,0.003864,0.004711,0.030907,0.001422,0.003988,0.003955,0.001618,0.004037


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2021-08-24      0.000034  0.001536  0.000005  0.000289  0.000042   
2021-08-25      0.000017  0.000768  0.000002  0.000145  0.000021   
2021-08-26      0.000009  0.007252  0.000001  0.000072  0.000010   

Province_State                                                                \
Admin2               Bullock        Butler   Calhoun  Chambers      Cherokee   
2021-08-24     -9.082613e-08  1.639652e-12  0.000772  0.000063  2.543134e-06   
2021-08-25     -4.541306e-08  8.198262e-13  0.000386  0.000031  1.271567e-06   
2021-08-26     -2.270653e-08  6.944444e-03  0.000193  0.003984  6.357835e-07   

Province_State  ...   Wyoming                                               \
Admin2          ...      Park        Platte  Sheridan  Sublette Sweetwater   
2021-08-24      ...  0.028572  1.702991e-08  0.000123  0.000434   0.032696   
2021-08-25      ...  0.014286  8.514954e-09  0.000062  0.000217   0.016348   
2021-08-26      ...  0.007143  4.257477e-09  0.000031  0.000109   0.008174   

Province_State                                                       \
Admin2                 Teton         Uinta Unassigned      Washakie   
2021-08-24      1.470543e-24  1.432804e-10   0.752941  1.556650e-63   
2021-08-25      7.352717e-25  7.164020e-11   0.752941  7.783251e-64   
2021-08-26      3.676358e-25  3.582010e-11   0.752941  3.891625e-64   

Province_State                
Admin2                Weston  
2021-08-24      1.323489e-24  
2021-08-25      6.617445e-25  
2021-08-26      3.308722e-25  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
